# Car Reviews

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import yfinance as yf
import seaborn as sns

import requests
from bs4 import BeautifulSoup
from time import time, sleep
import random
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
#declare header
headers = {"Accept-Language": "en-US,en;q=0.5"}

# Define the URL of the Reuters news page
url = "https://forums.edmunds.com/discussions/tagged/x/buying/p"
start_page = 1
end_page = 100

In [4]:
#data needed
title_list = []
views_list = []
comments_list = []
links_list = []

#page range
pages = np.arange(start_page,end_page+1)

#BS4 extraction
for page in pages:
    page = requests.get(url+str(page))
    soup = BeautifulSoup(page.text, 'html.parser')
    threads = soup.findAll('div', attrs = {'class': 'ItemContent Discussion'})
    for thread in threads:
        title = thread.find('div', class_ = "Title").text.lstrip('\n').rstrip(' ')
        title_list.append(title)
       
        views = thread.find("span", class_ = 'MItem MCount ViewCount').text.lstrip('\n').rstrip(' ')
        views_list.append(views)
       
        comments = thread.find("span", class_ = 'MItem MCount CommentCount').text.lstrip('\n').rstrip(' ')
        comments_list.append(comments)

        link = thread.find('div', class_ = "Title").find('a', href=True).get('href')
        links_list.append(link)

In [5]:
# create dataframe
df_heads = pd.DataFrame({'title': title_list,
                         'views': views_list,
                         'comments': comments_list,
                         'links': links_list})

# year, make, model
df_heads['year'] = df_heads['title'].str.split(' ').str[0]
df_heads['make'] = df_heads['title'].str.split(' ').str[1]
df_heads['model'] = df_heads['title'].str.split(' ').str[2]

# comments count and views count
df_heads['commentscount'] = df_heads['comments'].str.split(' ').str[0].str.replace('K', '00').str.replace('M', '00000').str.replace('.', '').astype(int)
df_heads['viewscount'] = df_heads['views'].str.split(' ').str[0].str.replace('K', '00').str.replace('M', '00000').str.replace('.', '').astype(int)

df_heads

,title,views,comments,links,year,make,model,commentscount,viewscount
0,2020 Honda Accord Lease Deals and Prices,30.3K views,1.4K comments,https://forums.edmunds.com/discussion/58876/ho...,2020,Honda,Accord,1400,30300
1,2022 BMW X7 Lease Deals and Prices,7.6K views,244 comments,https://forums.edmunds.com/discussion/66863/bm...,2022,BMW,X7,244,7600
2,2022 RAM 1500 Lease Deals and Prices,18.6K views,1.4K comments,https://forums.edmunds.com/discussion/67158/ra...,2022,RAM,1500,1400,18600
3,2022 Jeep Grand Cherokee Lease Deals and Prices,27K views,1.7K comments,https://forums.edmunds.com/discussion/67279/je...,2022,Jeep,Grand,1700,2700
4,Mercedes Benz EQS Lease Deals and Prices,10.5K views,636 comments,https://forums.edmunds.com/discussion/67606/me...,Mercedes,Benz,EQS,636,10500
...,...,...,...,...,...,...,...,...,...
3385,What will it take for consumers to buy America...,1.2K views,1.6K comments,https://forums.edmunds.com/discussion/4712/for...,What,will,it,1600,1200
3386,Make Me a Better (Online) Car Salesman!,493 views,658 comments,https://forums.edmunds.com/discussion/4849/gen...,Make,Me,a,658,493
3387,Best Fictional Character Car Spokesperson,321 views,71 comments,https://forums.edmunds.com/discussion/4880/sub...,Best,Fictional,Character,71,321
3388,Toyota Avalon - Which Trim Line to Choose?,131 views,10 comments,https://forums.edmunds.com/discussion/15882/to...,Toyota,Avalon,-,10,131


In [6]:
df_heads.to_csv('01_df_heads.csv', index=False)

In [7]:
df_heads = pd.read_csv('01_df_heads.csv', index_col=0)
df_heads.head()

,views,comments,links,year,make,model,commentscount,viewscount
title,,,,,,,,
2020 Honda Accord Lease Deals and Prices,30.3K views,1.4K comments,https://forums.edmunds.com/discussion/58876/ho...,2020,Honda,Accord,1400,30300
2022 BMW X7 Lease Deals and Prices,7.6K views,244 comments,https://forums.edmunds.com/discussion/66863/bm...,2022,BMW,X7,244,7600
2022 RAM 1500 Lease Deals and Prices,18.6K views,1.4K comments,https://forums.edmunds.com/discussion/67158/ra...,2022,RAM,1500,1400,18600
2022 Jeep Grand Cherokee Lease Deals and Prices,27K views,1.7K comments,https://forums.edmunds.com/discussion/67279/je...,2022,Jeep,Grand,1700,2700
Mercedes Benz EQS Lease Deals and Prices,10.5K views,636 comments,https://forums.edmunds.com/discussion/67606/me...,Mercedes,Benz,EQS,636,10500


### Reviews - Comments

In [12]:
def get_data(start_index, end_index):
  df_tot = pd.DataFrame()

  for link in range(start_index, end_index):

      ## NAMES & PAGES
      #-------------------------------------------------------
      url_p = df_heads['links'][link]
      page = requests.get(url_p)
      soup = BeautifulSoup(page.text, 'html.parser')

      # Names
      groups = soup.findAll('span', attrs = {'itemprop': 'itemListElement'})
      groupsname = []
      for i,g in enumerate(groups):
        groupsname.append(g.find('span', attrs = {'itemprop': 'name'}).text)
      #print(groupsname)

      # Number of pages
      threads = soup.findAll('div', attrs = {'role': 'navigation'})
      nav_list = []
      for thread in threads:
          navig = thread.findAll('a')#.get('href')
          for nav in navig:
            nav_list.append(nav.get('href'))

      if nav_list[0]!='/discussions':
        pvec = np.zeros(len(nav_list)-3)
        for i in range(1,len(nav_list)-2):
          pvec[i-1] =  nav_list[i].replace(nav_list[0],'').replace('/p','')
        maxp = int(pvec.max())
      else:
        maxp = 1


      ## REVIEWS - COMMENTS
      #-------------------------------------------------------
      link_list = []
      group_list = []
      subgroup_list = []
      user_list = []
      role_list = []
      postcount_list = []
      datecreated_list = []
      hourcreated_list = []
      message_list = []
      reply_list = []
      nreview_list = []
      nv = 0

      threads = soup.findAll('div', attrs = {'class': 'Comment'})

      for thread in threads:
          link_list.append(link+1)
          group_list.append(groupsname[0])
          subgroup_list.append(('' if len(groupsname)<2 else groupsname[1]))
          nreview_list.append(nv+1)

          user = thread.find('span', attrs = {'class': 'Author'}).find('a').get('title')
          user_list.append(user)

          role = thread.find('span', attrs = {'class': 'MItem RoleTitle'})
          role = None if role==None else role.text
          role_list.append(role)

          postcount = int(thread.find('span', attrs = {'class': 'MItem PostCount'}).find('b').text.replace('<b>','').replace(',',''))
          postcount_list.append(postcount)

          datecreated = thread.find('span', attrs = {'class': 'MItem DateCreated'}).find('time').get('datetime')[:10]
          datecreated_list.append(datecreated)

          hourcreated = thread.find('span', attrs = {'class': 'MItem DateCreated'}).find('time').get('datetime')[11:19]
          hourcreated_list.append(hourcreated)

          message = thread.find('div', attrs = {'class': 'Item-Body'}).find('div', attrs = {'class': 'Message userContent'}).text#.find('p', class_='blockquote-line') #Message userContent
          message_list.append(message.replace('\n',''))

          reply = 'No' if thread.find('div', attrs = {'class': 'Item-Body'}).find('p', class_='blockquote-line') == None else 'Yes'
          reply_list.append(reply)
          nv += 1

      if maxp>1:
          for npage in range(2,maxp+1):
              page = requests.get(url_p+'/p'+str(npage))
              soup = BeautifulSoup(page.text, 'html.parser')

              threads = soup.findAll('div', attrs = {'class': 'Comment'})

              for thread in threads:
                  link_list.append(link+1)
                  group_list.append(groupsname[0])
                  subgroup_list.append(('' if len(groupsname)<2 else groupsname[1]))
                  nreview_list.append(nv+1)

                  user = thread.find('span', attrs = {'class': 'Author'}).find('a').get('title')
                  user_list.append(user)

                  role = thread.find('span', attrs = {'class': 'MItem RoleTitle'})
                  role = None if role==None else role.text
                  role_list.append(role)

                  postcount = int(thread.find('span', attrs = {'class': 'MItem PostCount'}).find('b').text.replace('<b>','').replace(',',''))
                  postcount_list.append(postcount)

                  datecreated = thread.find('span', attrs = {'class': 'MItem DateCreated'}).find('time').get('datetime')[:10]
                  datecreated_list.append(datecreated)

                  hourcreated = thread.find('span', attrs = {'class': 'MItem DateCreated'}).find('time').get('datetime')[11:19]
                  hourcreated_list.append(hourcreated)

                  message = thread.find('div', attrs = {'class': 'Item-Body'}).find('div', attrs = {'class': 'Message userContent'}).text#.find('p', class_='blockquote-line') #Message userContent
                  message_list.append(message.replace('\n',''))

                  reply = 'No' if thread.find('div', attrs = {'class': 'Item-Body'}).find('p', class_='blockquote-line') == None else 'Yes'
                  reply_list.append(reply)
                  nv += 1

      # DataFrame
      df_aux = pd.DataFrame({'page': link_list,
                            'group': group_list,
                            'subgroup': subgroup_list,
                            'nreview': nreview_list,
                            'user': user_list,
                            'role': role_list,
                            'postcount': postcount_list,
                            'datecreated': datecreated_list,
                            'hourcreated':hourcreated_list,
                            'message':message_list,
                            'reply': reply_list})

      df_tot = df_tot.append(df_aux)
      if (link+1)%50==0:
          print('NLink: {:>5}'.format(link+1))

  return df_tot


In [13]:
df_tot = get_data(0,1)
df_tot.to_csv('02_df_reviews_5.csv', index=False)


# We will use multithreading to speed up the process
# We want to call get_data() function for each rng of 100 links
# We will use 5 threads

import threading

def get_data_thread(start_index, end_index):
    df_tot = get_data(start_index, end_index)
    df_tot.to_csv('02_df_reviews_'+str(start_index)+'_'+str(end_index)+'.csv', index=False)

# We will use 5 threads
threads = []
for i in range(0, 3300, 100):
    t = threading.Thread(target=get_data_thread, args=(i, i+100))
    threads.append(t)
    t.start()

for t in threads:
    t.join()



In [14]:
df_tot

,page,group,subgroup,nreview,user,role,postcount,datecreated,hourcreated,message,reply
0,1,Honda,Honda Accord,1,chait6756,Member,3,2019-11-02,07:16:09,"Hello,Can I request for the numbers like resid...",No
1,1,Honda,Honda Accord,2,Jwalker714,Member,4,2019-11-02,15:09:05,"Hi there! May I please get the MF, residual an...",No
2,1,Honda,Honda Accord,3,kyfdx,Moderator,217095,2019-11-02,15:28:12,"chait6756 said:Hello,Can I request for the num...",Yes
3,1,Honda,Honda Accord,4,Crownedjules,Member,34,2019-11-06,01:21:10,"Looking for RV, MF and incentives on 2020 Acco...",No
4,1,Honda,Honda Accord,5,Michaell,Moderator,219539,2019-11-06,07:33:53,"Crownedjules said:Looking for RV, MF and incen...",Yes
...,...,...,...,...,...,...,...,...,...,...,...
1374,1,Honda,Honda Accord,1375,kyfdx,Moderator,217095,2021-05-18,19:34:04,YaseemTheDream said:Can I get the MF/Residual/...,Yes
1375,1,Honda,Honda Accord,1376,Cam25,Member,3,2021-05-19,22:39:00,Hi there!I'm interested in a 2021 Honda Accord...,No
1376,1,Honda,Honda Accord,1377,Michaell,Moderator,219539,2021-05-20,00:38:49,Cam25 said:Hi there!I'm interested in a 2021 H...,Yes
1377,1,Honda,Honda Accord,1378,wasp,Member,2,2023-04-14,02:12:43,"Hello, Can you provide RV & MF for the 2020 Ho...",No


## Separate Message from Replies

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df_reviews = pd.read_csv('02_df_reviews_1.csv')#.append(pd.read_csv('02_df_reviews_2.csv'))#.append(pd.read_csv('02_df_reviews_3.csv'))
df_reviews['datecreated'] = pd.to_datetime(df_reviews['datecreated'])
df_reviews['replaced_message'] = df_reviews['message']
df_reviews.head()

,page,group,subgroup,nreview,user,role,postcount,datecreated,hourcreated,message,reply,replaced_message
0,1,Mercedes-Benz,Mercedes-Benz EQS,1,Michaell,Moderator,219239,2021-12-15,01:37:49,"momoney232 said:Hello, I am trying to get the ...",Yes,"momoney232 said:Hello, I am trying to get the ..."
1,1,Mercedes-Benz,Mercedes-Benz EQS,2,guptamk07,Member,2,2021-12-15,04:58:19,Thank you for the reply. Can you let me know w...,No,Thank you for the reply. Can you let me know w...
2,1,Mercedes-Benz,Mercedes-Benz EQS,3,Michaell,Moderator,219239,2021-12-15,06:11:40,guptamk07 said:Thank you for the reply. Can yo...,Yes,guptamk07 said:Thank you for the reply. Can yo...
3,1,Mercedes-Benz,Mercedes-Benz EQS,4,Johndoeiv,Member,1,2021-12-18,00:31:59,What about the eqs 580? Do you have rates for ...,No,What about the eqs 580? Do you have rates for ...
4,1,Mercedes-Benz,Mercedes-Benz EQS,5,Michaell,Moderator,219239,2021-12-18,00:39:45,Johndoeiv said:What about the eqs 580? Do you ...,Yes,Johndoeiv said:What about the eqs 580? Do you ...


In [ ]:
# Define function to replace text of previous row on each row
def replace_text(row):
    if row.name == 0:
        return row['message']
    elif pd.isnull(df_reviews.loc[row.name,'message']) or pd.isnull(df_reviews.loc[row.name-1,'message']):
        return ''
    elif df_reviews.loc[row.name-1, 'message'][0]=='@':
        b = (10 if len(df_reviews.loc[row.name-1, 'message'])>10 else len(df_reviews.loc[row.name-1, 'message']))
        return df_reviews.loc[row.name, 'message'].split(df_reviews.loc[row.name-1, 'message'][-b:])[-1]
    else:
        a_message = df_reviews.loc[row.name-1,'message'].lstrip(' ').rstrip(' ')
        a_replaced_message = df_reviews.loc[row.name-1, 'replaced_message'].lstrip(' ').rstrip(' ')
        a_usersaid = ('@' + df_reviews.loc[row.name-1, 'user'] if df_reviews.loc[row.name, 'message'][0]=='@' else df_reviews.loc[row.name-1, 'user']) + ' said:'

        return df_reviews.loc[row.name, 'message'].replace(a_message,'').replace(a_replaced_message,'').replace(a_usersaid, '')

In [ ]:
# Apply function to create new column with replaced text
df_reviews['replaced_message'] = df_reviews.apply(lambda row: replace_text(row), axis=1)

In [ ]:
df_reviews[-20:]

,page,group,subgroup,nreview,user,role,postcount,datecreated,hourcreated,message,reply,replaced_message
21459,100,Toyota,Toyota Sienna,51,08gpws2k,Member,1,2023-03-13,02:50:26,"Can some one with a Toyota lease ""calculator"" ...",No,"Can some one with a Toyota lease ""calculator"" ..."
21460,100,Toyota,Toyota Sienna,52,Michaell,Moderator,219239,2023-03-13,03:17:40,08gpws2k said:Can some one with a Toyota lease...,Yes,What is your location?What is the selling pric...
21461,100,Toyota,Toyota Sienna,53,kellerexpress,Member,6,2023-03-18,17:46:26,"Hello, can I please get the MF, incentives and...",No,"Hello, can I please get the MF, incentives and..."
21462,100,Toyota,Toyota Sienna,54,kyfdx,Moderator,216892,2023-03-18,17:48:56,"kellerexpress said:Hello, can I please get the...",Yes,36/12.00225 MF and 63% residualNo lease incent...
21463,100,Toyota,Toyota Sienna,55,kellerexpress,Member,6,2023-03-18,17:57:35,"kyfdx said: kellerexpress said:Hello, can I pl...",Yes,Thank you! So if the dealer is quoting me a h...
21464,100,Toyota,Toyota Sienna,56,kyfdx,Moderator,216892,2023-03-18,18:41:50,kellerexpress said: kyfdx said: kellerexpress ...,Yes,Possibly marking up the money factor.
21465,100,Toyota,Toyota Sienna,57,BrMn,Member,1,2023-03-19,17:32:24,"Hi,Please help with MF and RV for 2023 Sienna ...",No,"Hi,Please help with MF and RV for 2023 Sienna ..."
21466,100,Toyota,Toyota Sienna,58,kyfdx,Moderator,216892,2023-03-19,18:09:29,"BrMn said:Hi,Please help with MF and RV for 20...",Yes,.00225 MF62%/63% residual for XLE/XSEWe don't...
21467,100,Toyota,Toyota Sienna,59,kodimus1,Member,1,2023-03-22,07:54:03,"Can you please provide the Residual, MF for th...",No,"Can you please provide the Residual, MF for th..."
21468,100,Toyota,Toyota Sienna,60,kyfdx,Moderator,216892,2023-03-22,12:30:55,kodimus1 said:Can you please provide the Resid...,Yes,FWD or AWD?
